## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")




In [2]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY", "your_openai_api_key_here")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY", "your_langchain_api_key_here")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT", "your_langchain_project_here")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY", "your_groq_api_key_here")
os.environ["LANGCHAIN_TRACING_V2"]="true"


In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021AE33CFAD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021AE363CDD0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Rehaman and I am a  student studying in the field of AI.")])

AIMessage(content="Hello Rehaman, it's nice to meet you! \n\nI'm glad to hear you're studying AI. It's a fascinating and rapidly evolving field. \n\nWhat aspects of AI are you most interested in? Perhaps we could discuss them further.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 30, 'total_tokens': 89, 'completion_time': 0.107272727, 'prompt_time': 0.002141025, 'queue_time': 0.24207731400000002, 'total_time': 0.109413752}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--d6f14e08-87d1-4553-9611-497e5763f554-0', usage_metadata={'input_tokens': 30, 'output_tokens': 59, 'total_tokens': 89})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Rehaman and I am a  student studying in the field of AI."),
        AIMessage(content="Hi Rehaman,\n\nIt's nice to meet you!\n\nAI is a fascinating field. What aspects of AI are you most interested in studying? \n\nI'm always eager to learn more about what people are doing in this space.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Rehaman and that you are a student studying in the field of AI.  😄 \n\nIs there anything else you'd like to tell me about yourself or your studies?  I'm happy to chat!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 104, 'total_tokens': 157, 'completion_time': 0.096363636, 'prompt_time': 0.005542386, 'queue_time': 0.249957714, 'total_time': 0.101906022}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--f9e13b5f-b89d-4645-9670-5b5ce0558a27-0', usage_metadata={'input_tokens': 104, 'output_tokens': 53, 'total_tokens': 157})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
!pip install langchain_community


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [9]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , i am a lawyer always filing cases on AI")],
    config=config
)

In [10]:
response.content

"That's interesting! As an AI, I can't provide legal advice, but I can offer some information that might be relevant to your work. \n\nWhat kind of cases are you filing? Are they related to:\n\n* **Bias in AI algorithms?**\n* **Intellectual property rights of AI-generated content?**\n* **Liability for AI-caused harm?**\n* **Privacy concerns related to AI data collection?**\n* **Regulation of AI development and deployment?**\n\nKnowing the specific area of AI law you're focusing on would allow me to provide more tailored and helpful information. \n\nFor example, I could share:\n\n* **Recent legal developments and case precedents**\n* **Relevant legislation and regulations**\n* **Ethical considerations in AI**\n* **Resources for legal professionals specializing in AI**\n\n\nLet me know how I can be of assistance!  \n\n"

In [11]:
with_message_history.invoke(
    [HumanMessage(content="what i do ?")],
    config=config,
)

AIMessage(content="It sounds like you're interested in exploring the legal implications of AI. That's a fascinating and rapidly evolving field! \n\nSince I can't give legal advice, I can't tell you exactly *what* to do. But I can offer some ideas for how you could contribute to this area of law:\n\n**1. Specialize in AI Law:** \n   - Deepen your knowledge of AI technologies and their legal implications. \n   - Consider taking specialized courses or certifications in AI law.\n   - Network with other lawyers and experts in the field.\n\n**2. Advocate for Ethical AI Development:**\n   - Work with organizations promoting responsible AI development and use.\n   - Lobby for legislation that protects against AI bias, discrimination, and misuse.\n   - Educate the public about the potential benefits and risks of AI.\n\n**3. Represent Clients in AI-Related Disputes:**\n   - Advise businesses on legal compliance related to AI systems.\n   - Represent individuals harmed by AI-powered technologies.

In [12]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to know!\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Tom Cruise")],
    config=config1
)
response.content

"Well hello there, Tom Cruise! It's a pleasure to meet you. \n\nIs there anything I can help you with today? Perhaps you'd like to discuss your latest movie, or maybe you want to explore some creative writing ideas?  🎬  \n\nLet me know what's on your mind! 😁\n"

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'You told me your name is Tom Cruise!  \n\nIs there anything else I can help you with, Mr. Cruise? 😊  \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [15]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [16]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Rehaman")]})

AIMessage(content="Hello Rehaman! \n\nIt's nice to meet you. How can I help you today?  \n\nI'm ready for your questions! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 32, 'total_tokens': 70, 'completion_time': 0.069090909, 'prompt_time': 0.002354814, 'queue_time': 0.253378724, 'total_time': 0.071445723}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--f32434cb-448b-49c1-8821-0aef72d69129-0', usage_metadata={'input_tokens': 32, 'output_tokens': 38, 'total_tokens': 70})

In [17]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [18]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Rehaman")],
    config=config
)

response

AIMessage(content="Hello Rehaman! It's nice to meet you.  😊\n\nI'm happy to help with any questions you have. Just ask away!  What can I do for you today? \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 32, 'total_tokens': 77, 'completion_time': 0.081818182, 'prompt_time': 0.002537473, 'queue_time': 0.24323595599999998, 'total_time': 0.084355655}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--ae15e1e1-868e-4e07-b2b8-b578f5cfe52a-0', usage_metadata={'input_tokens': 32, 'output_tokens': 45, 'total_tokens': 77})

In [19]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Rehaman.  I remember! 😊  How can I help you further?\n'

In [20]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [21]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Rehaman")],"language":"Hindi"})
response.content

'नमस्ते Rehaman! \n\nमुझे खुशी है कि आप मुझसे बात कर रहे हैं। आप क्या जानना चाहेंगे? मैं आपकी सहायता करने के लिए यहाँ हूँ।  😊\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [22]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"

)

In [23]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Rehaman")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते Rehaman!  \n\nआप कैसे हैं? मैं आपकी मदद करने के लिए तैयार हूँ। कोई भी सवाल पूछने में संकोच न करें। \n'

In [24]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [25]:
response.content

'आपका नाम Rehaman है।  😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [27]:
!pip install transformers


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

ImportError: Could not import transformers python package. This is needed in order to calculate get_token_ids. Please install it with `pip install transformers`.

In [30]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

ImportError: Could not import transformers python package. This is needed in order to calculate get_token_ids. Please install it with `pip install transformers`.

In [ ]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2" 😊  \n\n\n\n'

In [ ]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [ ]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [ ]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"